## GK2A

In [7]:
import os
from netCDF4 import Dataset
import numpy as np
from osgeo import gdal, osr
from pyproj import Transformer
import rasterio

def get_gk2a_wv_bt(ipixel):
    gain = -0.0108914673328399 #WV063
    offset = 44.1777038574218 #WV063
    c0 = -1.76279494011147
    c1 = 1.00414910562278
    c2 = -9.83310914319385e-07

    cval = 299792458.0
    kval =1.3806488e-23
    hval =  6.62606957e-34

    wv_wave =6.21
    wn = (10000 /wv_wave) * 100
    e1 = (2 * hval * cval * cval) * np.power(wn, 3)

    data = gain * ipixel + offset
    e2 = (data * 1e-5)
    t_eff = ((hval * cval / kval) * wn) / np.log((e1 / e2) + 1)
    bt_data = c0 + c1 * t_eff + c2 * t_eff * t_eff

    gsics_data = bt_data
    gsics_data = np.asarray(gsics_data)

    return gsics_data
    
nc_file = Dataset('gk2a_ami_le1b_wv063_fd020ge_202303210310.nc', 'a')

# Add global metadata
nc_file.setncattr('spatial_resolution', '2km at nadir')
nc_file.setncattr('project', 'GOES')

nc_file.close()

print("Metadata added successfully.")

# Define the origin and pixel size
origin_x, origin_y = -5505547.6175907375, 5505547.6175907375
pixel_width, pixel_height = 2002.017315487541055, -2002.017315487541055

# Convert netCDF to GeoTIFF
command_gk2a_1 = (
    'gdal_translate -ot float32 -a_srs "+proj=geos +lon_0=128.2 +h=35786023 +sweep=x +datum=WGS84" '
    '-unscale -co COMPRESS=deflate NETCDF:"./gk2a_ami_le1b_wv063_fd020ge_202303210310.nc":image_pixel_values gk2a_platecarree_wv.tif'
)
os.system(command_gk2a_1)

# Update the origin and pixel size using gdal_edit.py
command_gk2a_2 = (
    f'gdal_edit.py -a_ullr {origin_x} {origin_y} {origin_x + pixel_width * 5500} {origin_y + pixel_height * 5500} gk2a_platecarree_wv.tif'
)
os.system(command_gk2a_2)

print("GeoTIFF file has been successfully created with the specified origin and pixel size.")

command_gk2a_2 = 'gdalwarp -t_srs EPSG:4326 -dstnodata 32768 gk2a_platecarree_wv.tif gk2a_platecarree_wv_geo.tif'

os.system(command_gk2a_2)

tif_file = 'gk2a_platecarree_wv_geo.tif'
output_tif_file = 'processed_gk2a_platecarree_wv_geo.tif'

with rasterio.open(tif_file) as src:
    data = src.read(1)
    processed_data = get_gk2a_wv_bt(data)
    meta = src.meta.copy()
    
    meta.update(dtype=rasterio.float32, nodata=np.nan)

    # Write the processed data to a new file
    with rasterio.open(output_tif_file, 'w', **meta) as dst:
        dst.write(processed_data.astype(rasterio.float32), 1)

print(f"Processed GeoTIFF file saved as {output_tif_file}")

Metadata added successfully.
Input file size is 5500, 5500
0...10...20...30...40...50...60...70...80...90...100 - done.
GeoTIFF file has been successfully created with the specified origin and pixel size.
Processing gk2a_platecarree_wv.tif [1/1] : 0

ERROR 1: Too many points (529 out of 529) failed to transform, unable to compute output bounds.
Warning 1: Unable to compute source region for output window 1777,0,1777,3158, skipping.


Using internal nodata values (e.g. 65535) for image gk2a_platecarree_wv.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


/tmp/ipykernel_4045/759965938.py:25: RuntimeWarning: invalid value encountered in log
  t_eff = ((hval * cval / kval) * wn) / np.log((e1 / e2) + 1)


Processed GeoTIFF file saved as processed_gk2a_platecarree_wv_geo.tif


## Metsat

In [8]:
command_metsat_1 = 'gdal_translate -b 5 -ot float32 -unscale -CO COMPRESS=deflate "./MSG4-SEVI-MSG15-0100-NA-20230321031243.846000000Z-NA.nat" metsat_platecarree_wv.tif'
command_metsat_2 = 'gdalwarp -t_srs EPSG:4326 -dstnodata -999.0 metsat_platecarree_wv.tif metsat_platecarree_wv_geo.tif'

os.system(command_metsat_1)
os.system(command_metsat_2)

Input file size is 3712, 3712
0...10...20...30...40...50...60...70...80...90...100 - done.
Processing metsat_platecarree_wv.tif [1/1] : 0Using internal nodata values (e.g. 0) for image metsat_platecarree_wv.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


0

In [50]:
import numpy as np
import rasterio

# Constants
c = 299792458  # Speed of light in m/s
h = 6.62606957e-34  # Planck constant in J.s
k = 1.3806488e-23  # Boltzmann constant in J/K

C1 = 1.19104273e-5
C2 = 1.43877523

nu_c = 1596.080  # in cm^-1
alpha = 0.9959
beta = 2.0780  # in K

#gain = 0.03862197
#offset = -0.42422367 #0.03862197, offset = -1.96972038
gain = 0.00831811
#gain = 0.00631811
offset = -0.42422367

def process_and_save(input_file, output_file):
    with rasterio.open(input_file) as src:
        data = src.read(1)  # Read the first band
        data = (data * gain + offset)
        data[data <= 0] = np.nan
        
        # Apply the conversion formula
        data = ((C2 * nu_c) / np.log((1.0 / data) * C1 * nu_c ** 3 + 1.0))
        data = (data - beta) / alpha
        
        # Update metadata
        meta = src.meta.copy()
        meta.update(dtype=rasterio.float32, nodata=np.nan)

        # Write the processed data to a new file
        with rasterio.open(output_file, 'w', **meta) as dst:
            dst.write(data.astype(rasterio.float32), 1)

    print(f"Processed GeoTIFF file saved as {output_file}")

# Process the individual files
input_files = ['metsat_platecarree_wv_geo.tif']
processed_files = ['metsat_platecarree_wv_geo3.tif']

for input_file, output_file in zip(input_files, processed_files):
    process_and_save(input_file, output_file)


Processed GeoTIFF file saved as metsat_platecarree_wv_geo3.tif


## GEOS16

In [16]:
import os
from netCDF4 import Dataset
import numpy as np
from osgeo import gdal, osr
from pyproj import Transformer
import rasterio

def get_geos18_wv_bt(rad):
    planck_fk1 = 50343.5
    planck_fk2 = 2326.3
    planck_bc1 = 1.69185
    planck_bc2 = 0.99636 
    BT = ((planck_fk2 / (np.log((planck_fk1 / rad) + 1))) - planck_bc1) / planck_bc2
    gsics_data = np.asarray(BT)
    return gsics_data

command_g18_1 = 'gdal_translate -ot float32 -unscale -CO COMPRESS=deflate NETCDF:"./OR_ABI-L1b-RadF-M6C08_G18_s20230800310204_e20230800319512_c20230800319566.nc":Rad geos18_platecarree_wv.tif'
command_g18_2 = 'gdalwarp -t_srs EPSG:4326 -dstnodata -999.0 geos18_platecarree_wv.tif geos18_platecarree_wv_geo.tif'

os.system(command_g18_1)
os.system(command_g18_2)

Input file size is 5424, 5424
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 7061P x 2997L.
Processing geos18_platecarree_wv.tif [1/1] : 0

ERROR 1: Too many points (529 out of 529) failed to transform, unable to compute output bounds.
Warning 1: Unable to compute source region for output window 3530,0,1765,2997, skipping.


Using internal nodata values (e.g. 4095) for image geos18_platecarree_wv.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


0

In [23]:
tif_file = 'geos18_platecarree_wv_geo.tif'
output_tif_file = 'geos18_platecarree_wv_geo2.tif'

with rasterio.open(tif_file) as src:
    data = src.read(1)
    processed_data = get_geos18_wv_bt(data)
    processed_data[processed_data < 0] = np.nan
    meta = src.meta.copy()
    
    meta.update(dtype=rasterio.float32, nodata= np.nan)

    # Write the processed data to a new file
    with rasterio.open(output_tif_file, 'w', **meta) as dst:
        dst.write(processed_data.astype(rasterio.float32), 1)

print(f"Processed GeoTIFF file saved as {output_tif_file}")

Processed GeoTIFF file saved as geos18_platecarree_wv_geo2.tif


/tmp/ipykernel_2708/3222414093.py:13: RuntimeWarning: invalid value encountered in log
  BT = ((planck_fk2 / (np.log((planck_fk1 / rad) + 1))) - planck_bc1) / planck_bc2


## MERGE

In [1]:
import os
import subprocess
import rasterio
import numpy as np

# Define the input files and nodata value
input_files = ['processed_gk2a_platecarree_wv_geo.tif', 'metsat_platecarree_wv_geo3.tif', 'geos18_platecarree_wv_geo2.tif']

# Merge the reprojected files with nodata handling
merge_command = f'gdal_merge.py -ps 0.018 0.018 -o pray_pray_wv_pr2.tif ' + ' '.join(input_files)
subprocess.run(merge_command, shell=True)

print("Files have been successfully merged into abaa.tif")


0...10...20...30...40...50...60...70...80...90...100 - done.
Files have been successfully merged into abaa.tif


In [2]:
import os
import subprocess
import rasterio
import numpy as np

# Define the input files and nodata value
input_files = ['metsat_platecarree_wv_geo3.tif', 'geos18_platecarree_wv_geo2.tif', 'processed_gk2a_platecarree_wv_geo.tif']

merge_command = f'gdal_merge.py -ps 0.018 0.018 -o pray_pray_wv_pr3.tif ' + ' '.join(input_files)
subprocess.run(merge_command, shell=True)

print("Files have been successfully merged into abaa.tif")


0...10...20...30...40...50...60...70...80...90...100 - done.
Files have been successfully merged into abaa.tif


In [3]:
output_tif_file = 'final_wv.tif'
tif_file = 'pray_pray_wv_pr3.tif'

with rasterio.open(tif_file) as src:
    data1 = src.read(1)

tif_file = 'pray_pray_wv_pr2.tif'

with rasterio.open(tif_file) as src:
    data2 = src.read(1)
    data2[:, -2500:] = data1[:,-2500:]
    meta = src.meta.copy()
    
    meta.update(dtype=rasterio.float32, nodata=np.nan)

    # Write the processed data to a new file
    with rasterio.open(output_tif_file, 'w', **meta) as dst:
        dst.write(data2.astype(rasterio.float32), 1)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 23.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.1
    Uninstalling numpy-2.0.1:
      Successfully uninstalled numpy-2.0.1


In [8]:
!pip install netCDF4